In [2]:
import nbimporter

from heapq import heappop, heappush, heapify
from map import Map

from astar_with_timestamps import astar_timesteps


In [8]:
def list_pair_from_list_nodes(path):
    p_path = []
    for i, node in enumerate(path):
        pair = (node.i, node.j)
        p_path.append(pair)
    return p_path


def run_with_order(traj_prev, traj_stopped, start, goal, search_func, map: Map, learning_rate: 0, lower_time_limit=0, **kwargs):
    
    ca_table = CATable(traj_prev, traj_stopped)
    
    result = search_func(map, ca_table, start[0], start[1], goal[0], goal[1], 
                         lower_time_limit=lower_time_limit, 
                         learning_rate=learning_rate, **kwargs)
    
    if result[0]:
        goal = result[1]
        path = make_path(goal)[0]
        p_path = list_pair_from_list_nodes(path)
        steps = result[2]
        if steps > lower_time_limit:
            lower_time_limit = steps
        return True, p_path, steps, lower_time_limit
    else:
        return False, None, 0, lower_time_limit


def FixedOrderSearch(starts, goals, map:Map, *, order=None, search_func=astar_timesteps, **kwargs):
    traj = []
    traj_stopped = []
    if order == None:
        order = list(range(len(starts)))
    
    lower_time_limit = 0
    learning_rate = 0
    for agent in range(len(starts)):
        res, p_path, steps, lower_time_limit = run_with_order(traj, 
                                                              traj_stopped,
                                                              starts[order[agent]], 
                                                              goals[order[agent]], 
                                                              search_func=search_func, 
                                                              map=map,
                                                              learning_rate=learning_rate,
                                                              lower_time_limit=lower_time_limit, **kwargs)
        learning_rate = .5
        map.clear_tmp()
        if res:
            traj.append(p_path)
            traj_stopped.append((goals[order[agent]], lower_time_limit))
        else:
            print(f"agent {agent} has failed")
            # return False, None, float("+inf")
    return True, traj, lower_time_limit